In [1]:
%cd /Users/trancatkhanh/Downloads/GAME

/Users/trancatkhanh/Downloads/GAME


In [2]:
import numpy as np
from base.Splendor_v3.env import *
#0 Array bias trong mỗi trận
#1 Array bias sau 100k trận 
#2 Lưu số trận 
#3 lưu hệ số đã dùng 
#4 Tỷ lệ thắng khi đấu 10k trận 
#5 Trigger để train 100k trận 
#6 Đếm số lần yếu liên tiếp
#7 Lưu array bias mới
#8 State của trận đấu
#9 Action của trận đấu
#10 Reward của trận đấu
#11 Số trận lưu state
#12 Số trận thắng của hệ số mới 

perx = [np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((1,getActionSize())),np.zeros((1,1)),np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((1,2)),np.zeros((1,2)),np.zeros((1,1)),np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((200000,getStateSize())),np.zeros((200000,1)),np.zeros((200000,1)),np.zeros((1,1))]
                         #0                                                      #1                   #2              #3                                                            #4              #5               #6                #7                                                               #8                                   #9               #10                       #11         

In [52]:
@njit()
def p_bias(state,per):
    # Chuyển Mode gồm 3 mode mode 0 - train , mode 1 - test , mode 2 - train policy yếu ,mode 3 - test policy yếu,mode 4 - lưu state
    if per[4][0][0] > per[5][0][1] and per[4][0][1] == 1000:
      per[3][0] = np.copy(per[1][0]/np.max(per[1][0]))
      # Lưu lại kết quả thắng
      per[5][0][1] = 0 
      per[5][0][1] += (per[4][0][0])
      # Rest số trận thắng và số trận đấu 
      per[4][0][0] = 0
      per[4][0][1] = 0 
      # Chuyển sang train
      per[5][0][0] = 0 
      per[2][0][0] = 0
      per[1][0] = np.zeros((1,getActionSize()))
      # Reset số lần yếu liên tiếp
      per[6][0][0] = 0 
    elif per[5][0][1] > per[4][0][0] and per[4][0][1] == 1000:
      per[6][0][0] +=1 
      # Chuyển sang train
      per[5][0][0] = 0 
      per[2][0][0] = 0
      per[1][0] = np.zeros((1,getActionSize()))
      # Rest số trận thắng và số trận đấu 
      per[4][0][0] = 0
      per[4][0][1] = 0 
    if per[6][0][0] == 3 and per[11][0][0] < 200000 :
      per[5][0][0] = 4
      per[2][0][0] = 0
      per[4][0][0] = 0
      per[4][0][1] = 0 
      per[5][0][1] = 0 

    if per[11][0][0] == 200000:
      if per[4][0][0] > per[5][0][1] and per[4][0][1] == 1000:
        per[3][0] = np.copy(per[1][0]/np.max(per[1][0]))
        # Lưu lại kết quả thắng
        per[5][0][1] = 0 
        per[5][0][1] += (per[4][0][0])
        # Rest số trận thắng và số trận đấu 
        per[4][0][0] = 0
        per[4][0][1] = 0 
        # Chuyển sang train
        per[5][0][0] = 2 
        per[2][0][0] = 0
        per[1][0] = np.zeros((1,getActionSize()))

      elif per[5][0][1] > per[4][0][0] and per[4][0][1] == 1000:
        # Chuyển sang train
        per[5][0][0] = 2 
        per[2][0][0] = 0
        per[1][0] = np.zeros((1,getActionSize()))
        # Rest số trận thắng và số trận đấu 
        per[4][0][0] = 0
        per[4][0][1] = 0 
      
    # reset per[1][0] sau 10k trận
    if int(per[2][0][0]) == 10000 and per[11][0][0] < 200000:
      per[3][0] = np.copy(per[1][0]/np.max(per[1][0]))
      per[2][0][0] = 0
      per[5][0][0] = 1
    # reset per[1][0] sau 10k trận
    if int(per[2][0][0]) == 10000 and  per[11][0][0] == 200000:
      per[7][0] = np.copy(per[1][0]/np.max(per[1][0]))
      per[2][0][0] = 0
      per[5][0][0] = 2


    # Chế độ train 
    if int(per[2][0][0]) < 10000 and per[5][0][0] == 0  :
      actions = getValidActions(state)
      actions *= per[0][0]
      action = np.argmax(actions)
      if getReward(state) == 1 :
          per[1] += per[0][0]
      if getReward(state) == 0:
          per[0][0] = np.random.rand(getActionSize())

    # Chế độ test 
    elif int(per[2][0][0]) <1000  and per[5][0][0] == 1 : 
      list_action2 = getValidActions(state)
      action = np.argmax(list_action2*per[3][0])    
      if getReward(state) == 1:
        per[4][0][0]+=1
      if getReward(state) != -1:
        per[4][0][1]+=1  

    # Chế độ train policy yếu
    elif int(per[2][0][0]) < 10000 and per[5][0][0] == 2 and per[11][0][0] == 200000 :
      actions = getValidActions(state)
      # Check xem policy có yếu không 
      Check_yeu = check_weaker(per[8],per[10],np.mean(state))
      if Check_yeu == 1:
        actions *= per[7][0]
        action = np.argmax(actions)
      else:
        actions *= per[3][0]
        action = np.argmax(actions)
      if getReward(state) == 1 and Check_yeu == 1:
        per[1] += per[0][0]
      if getReward(state) == 0 and Check_yeu == 1:
        per[0][0] = np.random.rand(getActionSize())

    # chế độ test policy yếu
    elif int(per[2][0][0]) <1000 and  per[5][0][0] == 3 and per[11][0][0] == 200000:
      actions = getValidActions(state)
      # Check xem policy có yếu không 
      if check_weaker(per[8],per[10],np.mean(state)) == 1:
        actions *= per[7][0]
        action = np.argmax(actions)
      else:
        actions *= per[3][0]
        action = np.argmax(actions)

      if getReward(state) == 1:
        per[4][0][0]+=1
      if getReward(state) != -1:
        per[4][0][1]+=1  


    # Báo chuyển sang chế độ lưu state và action
    if per[5][0][0] == 4 and per[11][0][0] < 200000:
      list_action2 = getValidActions(state)
      action = np.argmax(list_action2*per[3][0]) 


    # Lưu state và action
    if getReward(state) != -2 and per[11][0][0] < 200000 and per[5][0][0] == 4:
        for i in range(getStateSize()):
          per[8][int(per[11][0][0])][i] += state[i]        
        per[9][int(per[11][0][0])][0] += int(action)
        per[10][int(per[11][0][0])][0] += int(getReward(state))
        per[11][0][0] += 1

    # đếm số ván đấu 
    if getReward(state) != -1 :
        per[2][0][0] += 1
    return action,per

win, bias = numba_main_2(p_bias, 200000,perx,0)

In [6]:
perx

[array([[0.94900992, 0.40977139, 0.07858101, 0.58209774, 0.68403631,
         0.92401723, 0.13734011, 0.5904795 , 0.69805685, 0.40462962,
         0.70454175, 0.73464741, 0.02097298, 0.97108613, 0.73560333]]),
 array([[10514.76952456, 10529.55803325, 10639.62142059, 10434.67135843,
         10566.66630314, 10448.94897331, 10546.91735148, 10553.55653341,
         10557.14596754, 10469.36782552, 10488.26117659, 10511.3954308 ,
         10583.21227856, 10601.7057745 , 10624.04469232]]),
 array([[8955.]]),
 array([[0.91955908, 0.96984015, 1.        , 0.98064523, 0.91634293,
         0.91563015, 0.90908202, 0.90863418, 0.90771886, 0.91469945,
         0.90298689, 0.89578282, 0.95053939, 0.91049627, 0.90138285]]),
 array([[0., 0.]]),
 array([[2., 0.]]),
 array([[3.]]),
 array([[0.9852591 , 0.97970469, 1.        , 0.9754359 , 0.9828443 ,
         0.97749908, 0.98786641, 0.97924964, 0.98311394, 0.97538141,
         0.98157335, 0.97908724, 0.9938992 , 0.9839412 , 0.99407309]]),
 array([[ 7.,  7

In [28]:
np.mean(perx[8][1])

1.391304347826087

In [25]:
pr = np.zeros((200000,3))
pr[200][0]

0.0

In [34]:
j = np.zeros((200000,2))
j[1][0]

0.0

In [50]:
import numpy as np
@njit
def check_weaker(state, reward,x):
    just = np.zeros((200000,2))
    for i in range(len(reward)):
        if reward[i] == -1:
            for j in range(i,len(reward)):
                if reward[j] != -1:
                    reward[i] = reward[j]
                    break
    # np.mean each array in the state and the corresponding reward
    for i in range(len(reward)):
        just[i][0] = np.mean(state[i])
        just[i][1] = (reward[i][0])
    # count the number of times each value of the state appears and the reward of each value of the state
    count = np.zeros((200000,3))
    for i in range(len(reward)):
        for j in range(len(count)):
            if just[i][0] == count[j][0]:
                count[j][1] += 1
                count[j][2] += just[i][1]
                break
            if count[j][0] == 0:
                count[j][0] = just[i][0]
                count[j][1] += 1
                count[j][2] += just[i][1]
                break
    # find the value the lowest average reward
    min = 0
    for i in range(len(count)):
        if count[i][2] != 0:
            min = count[i][2]/count[i][1]
            break
    if x == min:
        return 1
    else:
        return 0


In [48]:
 2.03500000e+03

2035.0

In [47]:
3.83600000e+03

3836.0

In [67]:
perx[8]

array([[ 2.,  2.,  6., ...,  1., 12.,  0.],
       [ 1.,  2.,  4., ...,  1., 12.,  0.],
       [ 0.,  0.,  2., ...,  1., 12.,  0.],
       ...,
       [ 5.,  1.,  2., ...,  1., 12.,  0.],
       [ 3.,  3.,  2., ...,  1., 12.,  0.],
       [ 4.,  1.,  3., ...,  1., 12.,  0.]])

In [52]:
perx[8]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       ...,
       [-2., -2., -2., ..., -2., -2., -2.],
       [-2., -2., -2., ..., -2., -2., -2.],
       [-2., -2., -2., ..., -2., -2., -2.]])

In [50]:
l[:-1,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1.])

In [30]:
perx[8]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 5.,  5.,  3., ...,  1., 12.,  0.],
       [ 3.,  1.,  1., ...,  1., 12.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1., 10.,  0.],
       [ 0.,  2.,  0., ...,  1.,  9.,  0.],
       [ 4.,  5.,  1., ...,  1.,  9.,  0.]])

In [75]:
perx[10]

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [74]:
perx[8]


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 7.,  7.,  7., ...,  1., 12.,  0.],
       [ 2.,  6.,  7., ...,  1., 12.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [72]:
import numba
import numpy as np

@numba.njit
def find_important_feature(state, reward):
  n_features = state.shape[1]
  feature_count = np.zeros(n_features, dtype=np.int64)
  feature_value_count = [{} for _ in range(n_features)]
  
  for i in range(n_features):
    
    unique_values, counts = np.unique(state[:, i])
    print(counts)
    feature_count[i] = np.sum(counts)
    
    for value, count in zip(unique_values, counts):
      if value not in feature_value_count[i]:
        feature_value_count[i][value] = {"count": 0, "reward_count": 0}
        
      feature_value_count[i][value]["count"] = count
      feature_value_count[i][value]["reward_count"] = np.sum(reward[state[:, i] == value])
  
  important_feature = None
  important_feature_value = None
  lowest_percentage = 1
  
  for i in range(n_features):
    for value, count_data in feature_value_count[i].items():
      count = count_data["count"]
      reward_count = count_data["reward_count"]
      percentage = reward_count / count
      
      if percentage < lowest_percentage:
        lowest_percentage = percentage
        important_feature_value = value
  
  return important_feature, important_feature_value, lowest_percentage
find_important_feature(perx[8], perx[10])


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<function sum at 0x7fa54077d8b0>) found for signature:
 
 >>> sum(float64)
 
There are 2 candidate implementations:
[1m      - Of which 2 did not match due to:
      Overload of function 'sum': File: numba/core/typing/npydecl.py: Line 378.
        With argument(s): '(float64)':[0m
[1m       No match.[0m
[0m
[0m[1mDuring: resolving callee type: Function(<function sum at 0x7fa54077d8b0>)[0m
[0m[1mDuring: typing of call at /var/folders/ds/4lcv95w11y7brm6dqhc04s240000gn/T/ipykernel_46954/2782407322.py (14)
[0m
[1m
File "../../../../var/folders/ds/4lcv95w11y7brm6dqhc04s240000gn/T/ipykernel_46954/2782407322.py", line 14:[0m
[1m<source missing, REPL/exec in use?>[0m


In [59]:
import numpy as np
def find_important_feature(state, reward):
  for i in range(len(reward)):
      if reward[i] == -1:
          for j in range(i,len(reward)):
              if reward[j] != -1:
                  reward[i] = reward[j]
                  break
  
  n_features = state.shape[1]
  feature_count = np.zeros(n_features, dtype=int)
  feature_value_count = [{} for _ in range(n_features)]
  
  for i in range(n_features):    
    unique_values, counts = np.unique(state[:, i], return_counts=True)
    feature_count[i] = np.sum(counts)
    
    for value, count in zip(unique_values, counts):
      if value not in feature_value_count[i]:
        feature_value_count[i][value] = {"count": 0, "reward_count": 0}
        
      feature_value_count[i][value]["count"] = count
      feature_value_count[i][value]["reward_count"] = np.sum(reward[state[:, i] == value])
  
  important_feature = None
  important_feature_value = None
  lowest_percentage = 1
  
  for i in range(n_features):
    for value, count_data in feature_value_count[i].items():
      count = count_data["count"]
      reward_count = count_data["reward_count"]
      percentage = reward_count / count
      
      if percentage < lowest_percentage:
        lowest_percentage = percentage
        important_feature = "feature_{}".format(i)
        important_feature_value = value
  print(important_feature)
  print(important_feature_value)
  print(lowest_percentage)
  return important_feature, important_feature_value, lowest_percentage
find_important_feature(perx[8], perx[10])


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in function array>) found for signature:
 
 >>> array(array(float64, 2d, C))
 
There are 4 candidate implementations:
[1m      - Of which 4 did not match due to:
      Overload in function '_OverloadWrapper._build.<locals>.ol_generated': File: numba/core/overload_glue.py: Line 131.
        With argument(s): '(array(float64, 2d, C))':[0m
[1m       Rejected as the implementation raised a specific error:
         TypingError: Failed in nopython mode pipeline (step: nopython frontend)
       [1m[1m[1mNo implementation of function Function(<intrinsic stub>) found for signature:
        
        >>> stub(array(float64, 2d, C))
        
       There are 2 candidate implementations:
       [1m  - Of which 2 did not match due to:
         Intrinsic in function 'stub': File: numba/core/overload_glue.py: Line 35.
           With argument(s): '(array(float64, 2d,

# Find the most important value of index in the state that when it appear the policy dont work well


In [ ]:
def find_most_importane_feature(perx):
    import pandas as pd

    df = pd.DataFrame(perx[8])
    df['action'] = perx[9]
    df['reward'] = perx[10]
    for i in range(len(df)):
        if df['reward'][i] == -1:
            for j in range(i,len(df)):
                if df['reward'][j] != -1:
                    df['reward'][i] = df['reward'][j]
                    break
    df = df[df['reward'] != -1]

    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier

    # Load the state-action dataset

    # Split the data into features (state properties) and target (outcome)
    features = df.drop("reward", axis=1)
    target = df["reward"]

    # Train a Random Forest Classifier on the data
    clf = RandomForestClassifier()
    clf.fit(features, target)

    # Get the feature importances
    importances = clf.feature_importances_

    # Find the property (x) of the state that is most important for the policy's performance
    x = features.columns[importances.argmax()]
    # Plot the top  feature importances 
    import matplotlib.pyplot as plt
    import numpy as np
    indices = np.argsort(importances)[::-1]
    plt.figure(figsize=(10, 5))
    plt.title("Feature importances")
    plt.bar(range(features.shape[1]), importances[indices], color="r", align="center")
    plt.xticks(range(features.shape[1]), features.columns[indices], rotation=90)
    plt.xlim([-1, features.shape[1]])
    plt.show()
    most_imp_feature = None
    lowest_percentage = 100
    print('feature importances',x)
    for i in df[x].unique():
        current_percentage = df[df[x] == i]['reward'].value_counts()[1]/df[df[x] == i]['reward'].value_counts().sum()
        if current_percentage < lowest_percentage:
            most_imp_feature = i
            lowest_percentage = current_percentage

    print("The value of the most important feature with the lowest percentage of value 1 in reward is:", most_imp_feature)#  all the below code in one function    
    return most_imp_feature
